In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sqlalchemy
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(rc={'figure.figsize':(16,10)})
import sqlalchemy
import pandas as pd
import numpy as np
import random
import statsmodels.api as sm
from scipy.stats import shapiro
from itertools import combinations
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import norm

In [2]:
def read_txt(file_name=str, format='txt'):
    with open(f'{file_name}.{format}') as f: 
        sql_query = f.read()
    return sql_query

def get_gbq_data(query):
    project = 'lentochka-9bb50' 
    credentials_bq = service_account.Credentials.from_service_account_file('lentochka.json', scopes=["https://www.googleapis.com/auth/cloud-platform"],)
    pandas_gbq.context.credentials = credentials_bq
    pandas_gbq.context.project = project
    gbq_data = pandas_gbq.read_gbq(query, progress_bar_type=True)
    return gbq_data

def run_gbq_query(query):
    credentials = service_account. \
    Credentials. \
    from_service_account_file('lentochka.json', scopes = ['https://www.googleapis.com/auth/cloud-platform',            
                                                                                       "https://www.googleapis.com/auth/drive",
                                                                                       "https://www.googleapis.com/auth/bigquery"])
    project = 'lentochka-9bb50' 
    client = bigquery.Client(credentials=credentials, project=project)
    query_job = client.query(query)
    try:
        results = query_job.result()
    except:
        pass
    #for row in results:
        #print(row)

def append_bq_data(data, table):
    credentials = service_account. \
        Credentials. \
        from_service_account_file('lentochka.json', scopes = ['https://www.googleapis.com/auth/cloud-platform',            
                                                                                           "https://www.googleapis.com/auth/drive",
                                                                                           "https://www.googleapis.com/auth/bigquery"])
    project = 'lentochka-9bb50' 
    client = bigquery.Client(credentials=credentials, project=project)
    
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        write_disposition = bigquery.WriteDisposition.WRITE_APPEND
    )

    job = client.load_table_from_dataframe(
        data, table, job_config=job_config
    )
    job.result()
    print(f"Load {len(data)} rows")

In [20]:
password = 'dimVuidBun1onCub'
login = 'a.al.kosheleva'

def postre_auth(login, password):
    postgresql_config = {
    'postgresql_prefix' : 'postgresql'
    , 'postgresql_host'   : '10.3.26.92' 
    , 'postgresql_user'   : f'{login}'
    , 'postgresql_pass'   : f'{password}'
    , 'postgresql_port'   : '5432'
    , 'postgresql_db'     : 'dwh'    
    }

    postgresql_connection_string  = "{prefix}://{user}:{passw}@{host}:{port}/{db}".format(
    prefix  =   postgresql_config['postgresql_prefix']    , host  =   postgresql_config['postgresql_host']
    , user  =   postgresql_config['postgresql_user']      , passw =   postgresql_config['postgresql_pass']
    , port  =   postgresql_config['postgresql_port']      , db    =   postgresql_config['postgresql_db']
    )
    postgresql_engine = sqlalchemy.create_engine(postgresql_connection_string)
    postgresql_conn   = postgresql_engine.connect()
    return postgresql_engine, postgresql_conn

postrgre_engine, mysql_conn = postre_auth(login, password)

In [21]:
#загружу кпп, столбец - buyer_id
#если в колонках есть buyer_id, он просто делает из них одну табличку
def get_groups(con, br, t = 'test.csv', c = 'control.csv'):
    if br == 2:
        str_table_name = 'dwh_datamart.crm_samples_utk_users'
    else:
        str_table_name = 'dwh_datamart.crm_samples_lo_users'
    tmp_query = f'select buyer_id, phone from {str_table_name}'
    if 'buyer_id' in pd.read_csv(t, sep = ';').columns and 'buyer_id' in pd.read_csv(c, sep = ';').columns:
        print('buyer_id in header')
        df_test = pd.read_csv(t, sep = ';')
        df_control = pd.read_csv(c, sep = ';')
        df_test['ab_group'] = 'test'
        df_control['ab_group'] = 'control'
        df_final = df_test[['buyer_id', 'ab_group']].append(df_control[['buyer_id', 'ab_group']]).reset_index().drop(columns = 'index')
        df_final = df_final.rename(columns = {'buyer_id':'db_buyer_id'})[['db_buyer_id', 'ab_group']].dropna().copy()
    else:
        print('buyer_id not in header')
        df_users = pd.read_sql(tmp_query, con).rename(columns={'buyer_id':'db_buyer_id'})
        df_users['db_buyer_id'] = df_users['db_buyer_id'].astype(int).astype(str)
        def get_extra_info(data):
            tmp_data = data.copy()
            for el in tmp_data.columns:
                if el == 'phone':
                    tmp_data[el] = tmp_data[el].astype(str)
                elif el == 'lo_buyer_id':
                    tmp_data[el] = tmp_data[el].astype(int).astype(str)
                    tmp_data = tmp_data.rename(columns={'lo_buyer_id':'buyer_id'})
                elif el == 'buyer_id':
                    tmp_data[el] = tmp_data[el].astype(int).astype(str)
            try:
                tmp_data = tmp_data.merge(df_users, how = 'left', on = 'phone').dropna().copy()[['db_buyer_id', 'phone']]
            except:
                tmp_data = tmp_data.rename(columns = {'buyer_id':'db_buyer_id'})[['db_buyer_id']].dropna().copy()
            return tmp_data
        df_test = get_extra_info(pd.read_csv(t))
        df_control = get_extra_info(pd.read_csv(c))
        df_test['ab_group'] = 'test'
        df_control['ab_group'] = 'control'
        df_final = df_test.append(df_control).reset_index().drop(columns = 'index')
    df_final['db_buyer_id'] = df_final['db_buyer_id'].astype(str)
    return df_final

In [22]:
def get_metrics(
    d_st, # дата начала эксперимента
    d_end, # дата окончания эксперимента
    con, # подключение к БД
    data, # данные групп
    exp_name # краткое наименование эксперимента
):
    
    # Расчет финансовых данных
    str_query = read_txt('financial_query').replace('{date_start}', d_st).replace('{date_end}', d_end)
    df_financial_tmp = pd.read_sql(str_query, con).fillna(0)
    df_financial_tmp['buyer_id'] = df_financial_tmp['buyer_id'].astype(str)
    df_result = data.rename(columns = {'db_buyer_id':'buyer_id'}).merge(
        df_financial_tmp,
        how = 'left',
        on = 'buyer_id'
    ).fillna(0).drop(columns = 'min_dt')
    df_result['cnt_users'] = 1
    
    # Расчет воронки
    str_bq_table_name = f'lentochka-9bb50.stage.crm_ab_tmp_{exp_name}'
    run_gbq_query(f'drop table if exists `{str_bq_table_name}`')
    append_bq_data(df_result, str_bq_table_name)
    #print(str_bq_table_name)
    str_query = read_txt('client_logs').replace('{date_start}', d_st).replace('{date_end}', d_end).replace('{log_table}', str_bq_table_name)
    #print(str_query)
    df_result = get_gbq_data(str_query)
    run_gbq_query(f'drop table if exists `{str_bq_table_name}`')
    
    return df_result
    
    

In [23]:
def get_ab_results(
    data, # данные с метриками
    mtr, # словарь метрик
    params, # доп параметры эксперимента для записи в таблицу
    use_bucketing = 0, # использовать бакетирование
    n_buckets = 200,
    a = 0.05, # уровень доверия альфа
    n_subsamples = 1000 # количество подвыборок при бустрапе
):
    import warnings
    warnings.filterwarnings('ignore')
    
    def divide_zero(i, j):
        if j == 0:
            res = 0
        else:
            res = i / j
        return res
    
    # бутстрап выборки:
    def bootstrap_2d_data(
        data, # данные для бутстрапирования
        n_samples, # количество подвыборок
        d_mtr, # словарь метрик
        t # тип агрегации данных ('bucket', 'buyer_id')
    ):
        dict_metric_values = {}
        for i in range(n_samples):
            list_filter = list(np.random.choice(data[t], size=len(data), replace=True))
            df_filter = pd.DataFrame(
                {t:list(np.random.choice(data[t], size=len(data), replace=True))}, index = None
            ).copy()
            df_filter = df_filter.merge(data, how = 'left', on = t)
            for key, value in d_mtr.items():
                str_mtr_name = key
                str_mtr_num = value[0]
                str_mtr_denum = value[1]
                mtr_value = divide_zero(df_filter[str_mtr_num].sum(), df_filter[str_mtr_denum].sum())
                try:
                    dict_metric_values[key].append(mtr_value)
                except:
                    dict_metric_values[key] = [mtr_value]
        return dict_metric_values

    # ДИ на бутстрапированных данных:
    def stat_intervals(stat, alpha):
        boundaries = np.percentile(stat, [100 * alpha / 2., 50, 100 * (1 - alpha / 2.)])
        return boundaries
    
    data_copy = data.copy()
    
    # бакетирование (опционально)
    if use_bucketing == 1:
        np.random.seed(1)
        data_copy['bucket'] = np.random.randint(0, n_buckets, len(data_copy))
        dict_columns = {}
        for el in data_copy.columns:
            if el in ['buyer_id', 'phone', 'ab_group', 'bucket']:
                pass
            else:
                dict_columns[el] = 'sum'
        data_copy = data_copy.pivot_table(
            index = ['bucket', 'ab_group'],
            aggfunc = dict_columns
        ).reset_index()
        str_bootstrap_type = 'bucket'
    else:
        str_bootstrap_type = 'buyer_id'
    
    # оценка эксперимента
    data_copy_t = data_copy.loc[data_copy['ab_group'] == 'test'].reset_index().drop(columns = 'index')
    data_copy_c = data_copy.loc[data_copy['ab_group'] == 'control'].reset_index().drop(columns = 'index')
    
    dict_test = bootstrap_2d_data(
        data = data_copy_t, # данные для бутстрапирования
        n_samples = n_subsamples, # количество подвыборок
        d_mtr = mtr, # словарь метрик
        t = str_bootstrap_type # тип агрегации данных ('bucket', 'buyer_id')
    )
    dict_control = bootstrap_2d_data(
        data = data_copy_c, # данные для бутстрапирования
        n_samples = n_subsamples, # количество подвыборок
        d_mtr = mtr, # словарь метрик
        t = str_bootstrap_type # тип агрегации данных ('bucket', 'buyer_id')
    )
    
    df_result = pd.DataFrame()
    for key, value in mtr.items():
        str_mtr_name = key
        str_mtr_num = value[0]
        str_mtr_denum = value[1]
        str_mtr_id = value[2]
        str_mtr_id_up = value[3]
        str_mtr_type = value[4]
        str_mtr_denum_type = value[5]
        int_num_sum_a = data_copy_c[str_mtr_num].sum()
        int_denum_sum_a = data_copy_c[str_mtr_denum].sum()
        int_num_sum_b = data_copy_t[str_mtr_num].sum()
        int_denum_sum_b = data_copy_t[str_mtr_denum].sum()
        int_mtr_a = round(int_num_sum_a / int_denum_sum_a, 5)
        int_mtr_b = round(int_num_sum_b / int_denum_sum_b, 5)
        int_upl = round(int_mtr_b - int_mtr_a, 5)
        int_upl_perc = round(int_mtr_b / int_mtr_a - 1, 5)
        list_a_means = dict_control[key]
        list_b_means = dict_test[key]
        list_uplifts_abs = np.array([i for i in map(lambda x: x[1] - x[0], zip(list_a_means, list_b_means))])
        list_uplifts_perc = np.array([i for i in map(lambda x: x[1] / x[0] - 1, zip(list_a_means, list_b_means))])
        ci_abs = stat_intervals(list_uplifts_abs, 0.05)
        ci_perc = stat_intervals(list_uplifts_perc, 0.05)
        # запись результатов
        df_result = df_result.append(
            pd.DataFrame({
                'mtr_name':[str_mtr_name],
                'mtr_id':[str_mtr_id],
                'mtr_id_up':[str_mtr_id_up],
                'mtr_type':[str_mtr_type],
                'mtr_denum_type':[str_mtr_denum_type],
                'uplift':[int_mtr_b - int_mtr_a],
                'uplift_perc':[int_mtr_b / int_mtr_a - 1],
                'ci_perc_low':[ci_perc[0]],
                'ci_perc_median':[ci_perc[1]],
                'ci_perc_up':[ci_perc[2]],
                'mtr_a':[int_mtr_a],
                'mtr_b':[int_mtr_b],
                'a_num':[int_num_sum_a],
                'a_denum':[int_denum_sum_a],
                'b_num':[int_num_sum_b],
                'b_denum':[int_denum_sum_b],
            })
        )
        
    df_result['date_send'] = params[0].strip()
    df_result['date_start'] = params[1].strip()
    df_result['date_end'] = params[2].strip()
    df_result['short_exp_name'] = params[3].strip()
    df_result['segment_short'] = params[4].strip()
    df_result['segment_full'] = params[5].strip()
    df_result['target'] = params[6].strip()
    df_result['text'] = params[7].strip()
    df_result['brand'] = params[8].strip()
    df_result['a_num'] = df_result['a_num'].astype(int)
    df_result['b_num'] = df_result['b_num'].astype(int)
    
    return df_result

In [63]:
dict_metrics = {
    'Конверсия в заказ':['made_order', 'cnt_users', '01.01', '01', 'Финансовые', 'Аудитория рассылки'],
    'Частота заказов':['cnt_orders', 'made_order', '01.02', '01', 'Финансовые', 'Аудитория рассылки'],
    'ARPU':['revenue', 'cnt_users', '01.03', '01', 'Финансовые', 'Аудитория рассылки'],
    'ARPPU':['revenue', 'has_check', '01.04', '01', 'Финансовые', 'Купившие'],
    'AMPU':['margin', 'cnt_users', '01.05', '01', 'Финансовые', 'Аудитория рассылки'],
    'AMPPU':['margin', 'has_check', '01.06', '01', 'Финансовые', 'Купившие'],
    'Средний чек':['revenue', 'cnt_orders_checks', '01.07', '01', 'Финансовые', 'Заказы'],
    'Средняя маржа на заказ':['margin', 'cnt_orders_checks', '01.08', '01', 'Финансовые', 'Заказы'],
    
    'Использование промокодов':['use_code', 'has_check', '02.01', '02', 'Промо', 'Купившие'],
    'Потрачено баллов':['bonus_spent', 'has_check', '02.02', '02', 'Промо', 'Купившие'],
    
    'Конверсия в логин':['is_logined', 'cnt_users', '03.01', '03', 'Воронки', 'Аудитория рассылки'],
    'Конверсия в добавление товара':['add_to_cart', 'cnt_users', '03.02', '03', 'Воронки', 'Аудитория рассылки'],
    'Конверсия в корзину':['view_cart', 'cnt_users', '03.03', '03', 'Воронки', 'Аудитория рассылки'],
    'Конверсия в чекаут':['view_checkout', 'cnt_users', '03.04', '03', 'Воронки', 'Аудитория рассылки'],
    'Конверсия в покупку':['complete_purchase', 'cnt_users', '03.05', '03', 'Воронки', 'Аудитория рассылки']
}

# 1. Загрузка выборки эксперимента в разрезе групп

In [64]:
#pd.read_csv('test.csv', sep=';').rename(columns = {'nr':'phone', 'баер':'buyer_id'}).to_csv('test.csv', sep = ';')

In [77]:
pd.read_csv('test.csv', sep = ';').head(3)

,buyer_id
0,103184522
1,104167211
2,101507159


In [78]:
pd.read_csv('control.csv', sep = ';').head(3)

,buyer_id
0,103414604
1,104925507
2,101780983


In [79]:
df_sample = get_groups(
    con = mysql_conn,
    br = 1,
    t = 'test.csv', 
    c = 'control.csv'
)

buyer_id in header


In [80]:
df_sample.head(3)

,db_buyer_id,ab_group
0,103184522,test
1,104167211,test
2,101507159,test


In [81]:
df_sample.groupby('ab_group')['db_buyer_id'].count()

ab_group
control    11687
test       11300
Name: db_buyer_id, dtype: int64

# 2. Сбор метрик

In [82]:
f_date_send = '2023-05-12'

f_start_date = '2023-05-12'
f_end_date = '2023-05-15'

f_exp_name = 'utk_exp_loyalty_test_1'

f_segment_short = '-'

f_segment = """
Клиенты которым начислили 400 баллов)
"""

f_target = 'усиление'

f_text = 'начислено 400 баллов с 12-15.05'

f_brand = 'УТК'

exp_params = [f_date_send, f_start_date, f_end_date, f_exp_name, f_segment_short, f_segment, f_target, f_text, f_brand]

In [83]:
tmp_df = get_metrics(
    d_st = f_start_date, # дата начала эксперимента
    d_end = f_end_date, # дата окончания эксперимента
    con = mysql_conn, # подключение к БД
    data = df_sample, # данные групп
    exp_name = f_exp_name # краткое наименование эксперимента
)

Load 22987 rows


In [84]:
tmp_df.head()

,buyer_id,ab_group,use_code,made_order,has_check,cnt_orders,cnt_orders_checks,revenue,margin,bonus_spent,cnt_users,is_logined,add_to_cart,view_cart,view_checkout,complete_purchase
0,103184522,test,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0
1,104167211,test,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0
2,101507159,test,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0
3,104394993,test,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0
4,101339948,test,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0


In [85]:
tmp_df.shape

(22987, 16)

In [86]:
df_ab_results = get_ab_results(
    data = tmp_df, # данные с метриками
    mtr = dict_metrics, # словарь метрик
    params = exp_params, # доп параметры эксперимента для записи в таблицу
    use_bucketing = 1, # использовать бакетирование
    n_buckets = 200, # кол-во бакетов
    a = 0.05, # уровень доверия альфа
    n_subsamples = 1000, # количество подвыборок при бустрапе
)

In [87]:
df_ab_results.head()

,mtr_name,mtr_id,mtr_id_up,mtr_type,mtr_denum_type,uplift,uplift_perc,ci_perc_low,ci_perc_median,ci_perc_up,...,b_denum,date_send,date_start,date_end,short_exp_name,segment_short,segment_full,target,text,brand
0,Конверсия в заказ,01.01,01,Финансовые,Аудитория рассылки,0.01946,2.553806,1.849125,2.554659,3.667042,...,11300.0,2023-05-12,2023-05-12,2023-05-15,utk_exp_loyalty_test_1,-,Клиенты которым начислили 400 баллов),усиление,начислено 400 баллов с 12-15.05,УТК
0,Частота заказов,01.02,01,Финансовые,Аудитория рассылки,-0.00533,-0.004993,-0.066073,-0.002896,0.056216,...,306.0,2023-05-12,2023-05-12,2023-05-15,utk_exp_loyalty_test_1,-,Клиенты которым начислили 400 баллов),усиление,начислено 400 баллов с 12-15.05,УТК
0,ARPU,01.03,01,Финансовые,Аудитория рассылки,27.12642,1.366041,0.829732,1.361150,2.258370,...,11300.0,2023-05-12,2023-05-12,2023-05-15,utk_exp_loyalty_test_1,-,Клиенты которым начислили 400 баллов),усиление,начислено 400 баллов с 12-15.05,УТК
0,ARPPU,01.04,01,Финансовые,Купившие,-843.74884,-0.323572,-0.421907,-0.319199,-0.214817,...,301.0,2023-05-12,2023-05-12,2023-05-15,utk_exp_loyalty_test_1,-,Клиенты которым начислили 400 баллов),усиление,начислено 400 баллов с 12-15.05,УТК
0,AMPU,01.05,01,Финансовые,Аудитория рассылки,2.14619,0.433371,0.009942,0.428702,1.046405,...,11300.0,2023-05-12,2023-05-12,2023-05-15,utk_exp_loyalty_test_1,-,Клиенты которым начислили 400 баллов),усиление,начислено 400 баллов с 12-15.05,УТК


In [88]:
#append_bq_data(df_ab_results, 'lentochka-9bb50.reports.crm_ab_results_v2')

In [89]:
df_ab_results.to_excel('df_ab_results_test1.xlsx')